In [40]:
# Import necessary packages to handle DataFrames, JSON files, and create NaN values (pandas, json, and numpy)
import pandas as pd
import json
#“Normalize” semi-structured JSON data into a flat table
from pandas.io.json import json_normalize

# Set iPython's max column width to 1000
pd.set_option('display.max_columns', 1000)
#import numpy
import numpy as np
#import matplotlib
import matplotlib.pyplot as plt
#importing regular expression 
import re
#remove warnings
import warnings
warnings.filterwarnings('ignore')


In [41]:
# Loading the dataframe into df
df = pd.read_json('disresort1.json',lines = True)

# View the head of df 
df.head()

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,geo,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,lang,place,possibly_sensitive,quote_count,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,reply_count,retweet_count,retweeted,retweeted_status,source,text,timestamp_ms,truncated,user
0,NaN,None,2019-09-24 03:24:13,NaN,"{'hashtags': [{'text': 'TDR_now', 'indices': [...","{'media': [{'id': 1176330997974982657, 'id_str...",NaN,0,False,low,None,1176336509575712768,1176336509575712768,None,NaN,NaN,NaN,NaN,False,ja,None,0.0,0,NaN,NaN,NaN,NaN,0,0,False,{'created_at': 'Tue Sep 24 03:02:22 +0000 2019...,"<a href=""http://twitter.com/download/iphone"" r...",RT @tdrblognote: これが新しいミッキー花壇ですか～ 分離型です。イベントフォ...,2019-09-24 03:24:13.815,False,"{'id': 2228171312, 'id_str': '2228171312', 'na..."
1,NaN,None,2019-09-24 03:24:16,"[15, 140]","{'hashtags': [], 'urls': [{'url': 'https://t.c...",NaN,{'full_text': '@OrangeGrove55 Galaxy’s Edge at...,0,False,low,None,1176336519969345536,1176336519969345536,OrangeGrove55,1.176260e+18,1.176260e+18,8.504104e+17,8.504104e+17,False,en,None,NaN,0,NaN,NaN,NaN,NaN,0,0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@OrangeGrove55 Galaxy’s Edge at Walt Disney Wo...,2019-09-24 03:24:16.293,True,"{'id': 286969280, 'id_str': '286969280', 'name..."
2,NaN,None,2019-09-24 03:24:19,NaN,"{'hashtags': [{'text': 'TDR_now', 'indices': [...","{'media': [{'id': 1176294370477142021, 'id_str...",NaN,0,False,low,None,1176336531339919360,1176336531339919360,None,NaN,NaN,NaN,NaN,False,ja,None,0.0,0,NaN,NaN,NaN,NaN,0,0,False,{'created_at': 'Tue Sep 24 00:36:50 +0000 2019...,"<a href=""http://twitter.com/download/iphone"" r...",RT @hp_yec: 昨日となんか違うと思ったら\nミッキー花壇帰ってました！\nお帰り！...,2019-09-24 03:24:19.004,False,"{'id': 1573884836, 'id_str': '1573884836', 'na..."
3,NaN,None,2019-09-24 03:24:25,NaN,"{'hashtags': [{'text': 'Halloween', 'indices':...",NaN,NaN,0,False,low,None,1176336557969567744,1176336557969567744,None,NaN,NaN,NaN,NaN,False,en,None,NaN,0,NaN,NaN,NaN,NaN,0,0,False,{'created_at': 'Tue Sep 24 00:47:20 +0000 2019...,"<a href=""http://twitter.com/download/iphone"" r...",RT @gourmetdyy: New #Halloween themed cups! #S...,2019-09-24 03:24:25.353,False,"{'id': 181434711, 'id_str': '181434711', 'name..."
4,NaN,None,2019-09-24 03:24:28,NaN,"{'hashtags': [{'text': 'StarWars', 'indices': ...",NaN,NaN,0,False,low,None,1176336572305874944,1176336572305874944,None,NaN,NaN,NaN,NaN,True,en,None,NaN,0,{'created_at': 'Mon Sep 23 19:30:59 +0000 2019...,1.176217e+18,1.176217e+18,"{'url': 'https://t.co/WSTNWb85Za', 'expanded':...",0,0,False,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Why would Disney need to promote #StarWars: #G...,2019-09-24 03:24:28.771,False,"{'id': 73718254, 'id_str': '73718254', 'name':..."


In [24]:
#look at the columns we can use
print(df.columns)

Index(['contributors', 'coordinates', 'created_at', 'display_text_range',
       'entities', 'extended_entities', 'extended_tweet', 'favorite_count',
       'favorited', 'filter_level', 'geo', 'id', 'id_str',
       'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place',
       'possibly_sensitive', 'quote_count', 'quoted_status',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status_permalink',
       'reply_count', 'retweet_count', 'retweeted', 'retweeted_status',
       'source', 'text', 'timestamp_ms', 'truncated', 'user'],
      dtype='object')


In [42]:
#lets look at some of the nested dictionaries 
#entites
#extended_entities 
#extended_tweet
#retweeted_status
#user

In [43]:
#take columns for the sentiment analysis 
df1 = df[['text','id','lang','created_at',
          'user','source','retweeted_status',
         'display_text_range', 'extended_tweet', 'in_reply_to_status_id',
         'in_reply_to_status_id',  'in_reply_to_screen_name', 'extended_entities',
         'timestamp_ms', 'entities']]

df1.assign(text_length=df1['text'].apply(len))
print(df1.shape)

(1118, 15)


In [47]:
#lets look at our new selected columns 
print(df1.head(10))

                                                text                   id  \
0  RT @tdrblognote: これが新しいミッキー花壇ですか～ 分離型です。イベントフォ...  1176336509575712768   
1  @OrangeGrove55 Galaxy’s Edge at Walt Disney Wo...  1176336519969345536   
2  RT @hp_yec: 昨日となんか違うと思ったら\nミッキー花壇帰ってました！\nお帰り！...  1176336531339919360   
3  RT @gourmetdyy: New #Halloween themed cups! #S...  1176336557969567744   
4  Why would Disney need to promote #StarWars: #G...  1176336572305874944   
5  RT @tdrblognote: これが新しいミッキー花壇ですか～ 分離型です。イベントフォ...  1176336637992681472   
6  RT @MezzoMikiD: 東京ディズニーランド\n本日よりリニューアルしたミッキー花壇...  1176336652878303232   
7  You could win a vacation for four to a galaxy ...  1176336658029002752   
8  RT @_erikagirl00: a bitch just wants to go to ...  1176336672872517633   
9  RT @EricMMrozek: Why would Disney need to prom...  1176336681152208896   

  lang          created_at                                               user  \
0   ja 2019-09-24 03:24:13  {'id': 2228171312, 'id_str': '2228171312', 

In [49]:
#look at entities
df1.entities
for k, v in df1.entities.items():
    for k1, v1 in v.items():
        print(k1)

hashtags
urls
user_mentions
symbols
media
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
media
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
media
hashtags
urls
user_mentions
symbols
media
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
media
hashtags
urls
user_mentions
symbols
media
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
media
hashtags
urls
user_mentions
symbols
media
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
media
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
hashtags
u

symbols
hashtags
urls
user_mentions
symbols
media
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
media
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
media
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
media
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
media
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mentions
symbols
hashtags
urls
user_mention

In [50]:
#look into retweeted_status
df1.retweeted_status
for k, v in df1.retweeted_status.items():
    for k1, v1 in v.items():
        print(k1)

created_at
id
id_str
text
display_text_range
source
truncated
in_reply_to_status_id
in_reply_to_status_id_str
in_reply_to_user_id
in_reply_to_user_id_str
in_reply_to_screen_name
user
geo
coordinates
place
contributors
is_quote_status
quote_count
reply_count
retweet_count
favorite_count
entities
extended_entities
favorited
retweeted
possibly_sensitive
filter_level
lang


AttributeError: 'float' object has no attribute 'items'

In [52]:
#look at extended_tweet
df1.extended_tweet
for k, v in df1.extended_tweet.items():
    for k1, v1 in v.items():
        print(k1)

AttributeError: 'float' object has no attribute 'items'

In [53]:
#look at user
df1.user
for k, v in df1.user.items():
    for k1, v1 in v.items():
        print(k1)

id
id_str
name
screen_name
location
url
description
translator_type
protected
verified
followers_count
friends_count
listed_count
favourites_count
statuses_count
created_at
utc_offset
time_zone
geo_enabled
lang
contributors_enabled
is_translator
profile_background_color
profile_background_image_url
profile_background_image_url_https
profile_background_tile
profile_link_color
profile_sidebar_border_color
profile_sidebar_fill_color
profile_text_color
profile_use_background_image
profile_image_url
profile_image_url_https
profile_banner_url
default_profile
default_profile_image
following
follow_request_sent
notifications
id
id_str
name
screen_name
location
url
description
translator_type
protected
verified
followers_count
friends_count
listed_count
favourites_count
statuses_count
created_at
utc_offset
time_zone
geo_enabled
lang
contributors_enabled
is_translator
profile_background_color
profile_background_image_url
profile_background_image_url_https
profile_background_tile
profile_link_col

listed_count
favourites_count
statuses_count
created_at
utc_offset
time_zone
geo_enabled
lang
contributors_enabled
is_translator
profile_background_color
profile_background_image_url
profile_background_image_url_https
profile_background_tile
profile_link_color
profile_sidebar_border_color
profile_sidebar_fill_color
profile_text_color
profile_use_background_image
profile_image_url
profile_image_url_https
profile_banner_url
default_profile
default_profile_image
following
follow_request_sent
notifications
id
id_str
name
screen_name
location
url
description
translator_type
protected
verified
followers_count
friends_count
listed_count
favourites_count
statuses_count
created_at
utc_offset
time_zone
geo_enabled
lang
contributors_enabled
is_translator
profile_background_color
profile_background_image_url
profile_background_image_url_https
profile_background_tile
profile_link_color
profile_sidebar_border_color
profile_sidebar_fill_color
profile_text_color
profile_use_background_image
profile_im

location
url
description
translator_type
protected
verified
followers_count
friends_count
listed_count
favourites_count
statuses_count
created_at
utc_offset
time_zone
geo_enabled
lang
contributors_enabled
is_translator
profile_background_color
profile_background_image_url
profile_background_image_url_https
profile_background_tile
profile_link_color
profile_sidebar_border_color
profile_sidebar_fill_color
profile_text_color
profile_use_background_image
profile_image_url
profile_image_url_https
profile_banner_url
default_profile
default_profile_image
following
follow_request_sent
notifications
id
id_str
name
screen_name
location
url
description
translator_type
protected
verified
followers_count
friends_count
listed_count
favourites_count
statuses_count
created_at
utc_offset
time_zone
geo_enabled
lang
contributors_enabled
is_translator
profile_background_color
profile_background_image_url
profile_background_image_url_https
profile_background_tile
profile_link_color
profile_sidebar_border_c

profile_image_url_https
profile_banner_url
default_profile
default_profile_image
following
follow_request_sent
notifications
id
id_str
name
screen_name
location
url
description
translator_type
protected
verified
followers_count
friends_count
listed_count
favourites_count
statuses_count
created_at
utc_offset
time_zone
geo_enabled
lang
contributors_enabled
is_translator
profile_background_color
profile_background_image_url
profile_background_image_url_https
profile_background_tile
profile_link_color
profile_sidebar_border_color
profile_sidebar_fill_color
profile_text_color
profile_use_background_image
profile_image_url
profile_image_url_https
profile_banner_url
default_profile
default_profile_image
following
follow_request_sent
notifications
id
id_str
name
screen_name
location
url
description
translator_type
protected
verified
followers_count
friends_count
listed_count
favourites_count
statuses_count
created_at
utc_offset
time_zone
geo_enabled
lang
contributors_enabled
is_translator
prof

profile_banner_url
default_profile
default_profile_image
following
follow_request_sent
notifications
id
id_str
name
screen_name
location
url
description
translator_type
protected
verified
followers_count
friends_count
listed_count
favourites_count
statuses_count
created_at
utc_offset
time_zone
geo_enabled
lang
contributors_enabled
is_translator
profile_background_color
profile_background_image_url
profile_background_image_url_https
profile_background_tile
profile_link_color
profile_sidebar_border_color
profile_sidebar_fill_color
profile_text_color
profile_use_background_image
profile_image_url
profile_image_url_https
profile_banner_url
default_profile
default_profile_image
following
follow_request_sent
notifications
id
id_str
name
screen_name
location
url
description
translator_type
protected
verified
followers_count
friends_count
listed_count
favourites_count
statuses_count
created_at
utc_offset
time_zone
geo_enabled
lang
contributors_enabled
is_translator
profile_background_color
pro

...

is_translator
profile_background_color
profile_background_image_url
profile_background_image_url_https
profile_background_tile
profile_link_color
profile_sidebar_border_color
profile_sidebar_fill_color
profile_text_color
profile_use_background_image
profile_image_url
profile_image_url_https
profile_banner_url
default_profile
default_profile_image
following
follow_request_sent
notifications
id
id_str
name
screen_name
location
url
description
translator_type
protected
verified
followers_count
friends_count
listed_count
favourites_count
statuses_count
created_at
utc_offset
time_zone
geo_enabled
lang
contributors_enabled
is_translator
profile_background_color
profile_background_image_url
profile_background_image_url_https
profile_background_tile
profile_link_color
profile_sidebar_border_color
profile_sidebar_fill_color
profile_text_color
profile_use_background_image
profile_image_url
profile_image_url_https
default_profile
default_profile_image
following
follow_request_sent
notifications
i

profile_background_image_url_https
profile_background_tile
profile_link_color
profile_sidebar_border_color
profile_sidebar_fill_color
profile_text_color
profile_use_background_image
profile_image_url
profile_image_url_https
profile_banner_url
default_profile
default_profile_image
following
follow_request_sent
notifications
id
id_str
name
screen_name
location
url
description
translator_type
protected
verified
followers_count
friends_count
listed_count
favourites_count
statuses_count
created_at
utc_offset
time_zone
geo_enabled
lang
contributors_enabled
is_translator
profile_background_color
profile_background_image_url
profile_background_image_url_https
profile_background_tile
profile_link_color
profile_sidebar_border_color
profile_sidebar_fill_color
profile_text_color
profile_use_background_image
profile_image_url
profile_image_url_https
default_profile
default_profile_image
following
follow_request_sent
notifications
id
id_str
name
screen_name
location
url
description
translator_type
p

geo_enabled
lang
contributors_enabled
is_translator
profile_background_color
profile_background_image_url
profile_background_image_url_https
profile_background_tile
profile_link_color
profile_sidebar_border_color
profile_sidebar_fill_color
profile_text_color
profile_use_background_image
profile_image_url
profile_image_url_https
profile_banner_url
default_profile
default_profile_image
following
follow_request_sent
notifications
id
id_str
name
screen_name
location
url
description
translator_type
protected
verified
followers_count
friends_count
listed_count
favourites_count
statuses_count
created_at
utc_offset
time_zone
geo_enabled
lang
contributors_enabled
is_translator
profile_background_color
profile_background_image_url
profile_background_image_url_https
profile_background_tile
profile_link_color
profile_sidebar_border_color
profile_sidebar_fill_color
profile_text_color
profile_use_background_image
profile_image_url
profile_image_url_https
profile_banner_url
default_profile
default_pr

profile_banner_url
default_profile
default_profile_image
following
follow_request_sent
notifications
id
id_str
name
screen_name
location
url
description
translator_type
protected
verified
followers_count
friends_count
listed_count
favourites_count
statuses_count
created_at
utc_offset
time_zone
geo_enabled
lang
contributors_enabled
is_translator
profile_background_color
profile_background_image_url
profile_background_image_url_https
profile_background_tile
profile_link_color
profile_sidebar_border_color
profile_sidebar_fill_color
profile_text_color
profile_use_background_image
profile_image_url
profile_image_url_https
profile_banner_url
default_profile
default_profile_image
following
follow_request_sent
notifications
id
id_str
name
screen_name
location
url
description
translator_type
protected
verified
followers_count
friends_count
listed_count
favourites_count
statuses_count
created_at
utc_offset
time_zone
geo_enabled
lang
contributors_enabled
is_translator
profile_background_color
pro

profile_banner_url
default_profile
default_profile_image
following
follow_request_sent
notifications
id
id_str
name
screen_name
location
url
description
translator_type
protected
verified
followers_count
friends_count
listed_count
favourites_count
statuses_count
created_at
utc_offset
time_zone
geo_enabled
lang
contributors_enabled
is_translator
profile_background_color
profile_background_image_url
profile_background_image_url_https
profile_background_tile
profile_link_color
profile_sidebar_border_color
profile_sidebar_fill_color
profile_text_color
profile_use_background_image
profile_image_url
profile_image_url_https
profile_banner_url
default_profile
default_profile_image
following
follow_request_sent
notifications
id
id_str
name
screen_name
location
url
description
translator_type
protected
verified
followers_count
friends_count
listed_count
favourites_count
statuses_count
created_at
utc_offset
time_zone
geo_enabled
lang
contributors_enabled
is_translator
profile_background_color
pro

In [55]:
#look at extended_entities
df1.extended_entities
for k, v in df1.extended_entities():
    for k1, v1 in v.items():
        print(k1)

TypeError: 'Series' object is not callable

In [56]:
#selecting dictionaries from user, nested dictionary
df1['user_id'] = df1['user'].transform(lambda x: x['id'])
df1['user_screen_name'] = df1['user'].transform(lambda x: x['screen_name'])
df1['user_location'] = df1['user'].transform(lambda x: x['location'])
df1['user_description'] = df1['user'].transform(lambda x: x['description'])
df1['user_followers_count'] = df1['user'].transform(lambda x: x['followers_count'])
df1['user_friends_count'] = df1['user'].transform(lambda x: x['friends_count'])
df1['user_favourites_count'] = df1['user'].transform(lambda x: x['favourites_count'])
df1['user_statuses_count'] = df1['user'].transform(lambda x: x['statuses_count'])
df1['user_created_at'] = df1['user'].transform(lambda x: x['created_at'])
df1['user_listed_count'] = df1['user'].transform(lambda x: x['listed_count'])

#drop users column
df1=df1.drop(columns='user')

In [57]:
print(df1.head(5))
print(df1.shape)

                                                text                   id  \
0  RT @tdrblognote: これが新しいミッキー花壇ですか～ 分離型です。イベントフォ...  1176336509575712768   
1  @OrangeGrove55 Galaxy’s Edge at Walt Disney Wo...  1176336519969345536   
2  RT @hp_yec: 昨日となんか違うと思ったら\nミッキー花壇帰ってました！\nお帰り！...  1176336531339919360   
3  RT @gourmetdyy: New #Halloween themed cups! #S...  1176336557969567744   
4  Why would Disney need to promote #StarWars: #G...  1176336572305874944   

  lang          created_at                                             source  \
0   ja 2019-09-24 03:24:13  <a href="http://twitter.com/download/iphone" r...   
1   en 2019-09-24 03:24:16  <a href="http://twitter.com/download/iphone" r...   
2   ja 2019-09-24 03:24:19  <a href="http://twitter.com/download/iphone" r...   
3   en 2019-09-24 03:24:25  <a href="http://twitter.com/download/iphone" r...   
4   en 2019-09-24 03:24:28  <a href="https://mobile.twitter.com" rel="nofo...   

                                    retweeted_stat

In [39]:
#selecting dictionaries from retweeted_status, nested dictionary
df1['retweeted_status_id'] = df1['retweeted_status'].transform(lambda x: x['id'])
df1['retweeted_status_screen_name'] = df1['retweeted_status'].transform(lambda x: x['screen_name'])
df1['retweeted_status_location'] = df1['retweeted_status'].transform(lambda x: x['location'])
df1['retweeted_status_description'] = df1['retweeted_status'].transform(lambda x: x['description'])
df1['retweeted_status_followers_count'] = df1['retweeted_status'].transform(lambda x: x['followers_count'])
df1['retweeted_status_friends_count'] = df1['retweeted_status'].transform(lambda x: x['friends_count'])
df1['retweeted_status_favourites_count'] = df1['retweeted_status'].transform(lambda x: x['favourites_count'])
df1['retweeted_status_statuses_count'] = df1['retweeted_status'].transform(lambda x: x['statuses_count'])
df1['retweeted_status_created_at'] = df1['retweeted_status'].transform(lambda x: x['created_at'])
df1['retweeted_status_listed_count'] = df1['retweeted_status'].transform(lambda x: x['listed_count'])
df1['retweeted_status_in_reply_to_screen_name'] = df1['retweeted_status'].transform(lambda x: x['in_reply_to_screen_name'])
df1['retweeted_status_source'] = df1['retweeted_status'].transform(lambda x: x['source'])
df1['retweeted_status_retweet_count'] = df1['retweeted_status'].transform(lambda x: x['retweet_count'])
df1['retweeted_status_retweeted'] = df1['retweeted_status'].transform(lambda x: x['retweeted'])
                                          
#drop retweeted_status column
df1=df1.drop(columns='retweeted_status')

KeyError: 'id'